In [1]:
import requests
import pandas as pd
import numpy as np
import requests
import bs4
import ssl
import certifi
import time

In [ ]:
forbes = pd.read_excel('forbes_1000.xlsx')

In [32]:
def grab_top_link(text):
    search = 'http://www.google.com/search?q=' + text
  
    # Fetch the URL data using requests.get(url),
    # store it in a variable, request_result.
    r = requests.get(search)

    soup = bs4.BeautifulSoup(r.text, "html.parser")

    result_div = soup.find_all('div', attrs = {'class': 'ZINbbc'})

    links = []
    titles = []
    descriptions = []
    for r in result_div:
        # Checks if each element is present, else, raise exception
        try:
            link = r.find('a', href = True)
            title = r.find('div', attrs={'class':'vvjwJb'}).get_text()
            description = r.find('div', attrs={'class':'s3v9rd'}).get_text()

            # Check to make sure everything is present before appending
            if link != '' and title != '' and description != '': 
                links.append(link['href'])
                titles.append(title)
                descriptions.append(description)
        # Next loop if one element is not present
        except:
            continue
    import re   

    to_remove = []
    clean_links = []
    for i, l in enumerate(links):
        clean = re.search('\/url\?q\=(.*)\&sa',l)

        # Anything that doesn't fit the above pattern will be removed
        if clean is None:
            to_remove.append(i)
            continue
        clean_links.append(clean.group(1))

    # Remove the corresponding titles & descriptions
    for x in to_remove:
        del titles[x]
        del descriptions[x]
        
    top_link = clean_links[0]
    i = 0
    wiki = 0
    while 'wikipedia' in top_link or 'history' in top_link:
        wiki += 1
        i+=1
        top_link = clean_links[i]
    return [top_link, wiki]

In [33]:
grab_top_link('aam')

['https://www.aam-us.org/', 0]

In [ ]:
company_good = {}
company_bad = []
last_req = time.time()
for company in forbes.Name:
    link = grab_top_link(company)
    if link[1] < 0:
        company_good[company] = link[1]
    else:
        company_good[company] = link[0]
    
    now = time.time()
    delay = last_req + 0.900 - now
    last_req = now
    if delay >= 0:
        time.sleep(delay)

In [ ]:
company_good

In [ ]:
company_bad